## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. 

In [1]:
# Подключение библиотек

import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator
        
RANDOM_SEED = 2022

In [2]:
# Инициализация локальной spark-сессии
spark = SparkSession.builder \
                    .master("local") \
                    .appName("housing") \
                    .getOrCreate()

In [3]:
# Чтение файла housing.csv

try:
    df = spark.read.option('header', 'true').csv('housing.csv', inferSchema = True)
except:
    df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)

In [4]:
# Вывод первых 5 строк датасета
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [5]:
# Типы данных колонок с помощью метода Pyspark
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



## Подготовка данных

In [6]:
# разделение столбцов на 3 категории
categorical_cols = ['ocean_proximity']
numerical_cols  = df.columns
numerical_cols.remove('ocean_proximity')
numerical_cols.remove('median_house_value')
target = 'median_house_value'

In [7]:
numerical_cols

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [8]:
# Количество строк датасета
df.count()

20640

In [9]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Пропуски есть только в колонке с количеством спален, заполним пропуски медианным значением

In [10]:
# Найдем медиану по количеству спален
median = df.select('total_bedrooms').toPandas().median()[0]

In [11]:
median

435.0

In [12]:
df = df.na.fill({'total_bedrooms': median})

In [13]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,536.8388565891473,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.39187792168866,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


In [14]:
# Проверка того, какие значения принимает единственный категориальный столбец - близость к океану
df.select('ocean_proximity').distinct().toPandas()

,ocean_proximity
0,ISLAND
1,NEAR OCEAN
2,NEAR BAY
3,<1H OCEAN
4,INLAND


Трансформируем данный признак


In [15]:
# Трансформация категориального признака с помощью трансформера StringIndexer

indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols]) 

df = indexer.fit(df).transform(df)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
df.select(cols).show(5)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 5 rows



In [16]:
# OHE кодирование

encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])

df = encoder.fit(df).transform(df)

cols = [c for c in df.columns for i in categorical_cols if (c.startswith(i))]
df.select(cols).show(5)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 5 rows



In [17]:
# Создание единого вектора категорий
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                                        outputCol="categorical_features")
df = categorical_assembler.transform(df)

Далее необходимо провести трансформацию числовых признаков

In [18]:
# Создание вектора объединенных числовых признаков

numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                      outputCol="numerical_features")

df = numerical_assembler.transform(df)

In [19]:
# Стандартизация числовых признаков

standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol="numerical_features_scaled")

df = standardScaler.fit(df).transform(df)

In [20]:
# Объединение трансформированных категорийных признаков и числовых

all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
df = final_assembler.transform(df)

df.select(all_features).show(5)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[3],[1.0])|     [-61.007269596069...|
|       (4,[3],[1.0])|     [-61.002278409814...|
|       (4,[3],[1.0])|     [-61.012260782324...|
|       (4,[3],[1.0])|     [-61.017251968579...|
|       (4,[3],[1.0])|     [-61.017251968579...|
+--------------------+-------------------------+
only showing top 5 rows



## Разделение на выборки

In [21]:
# Деление выборки в соответствии 80 на 20

train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

16418 4222


## Обучение моделей

Построение модели линейной регрессии на всех данных

In [22]:
# Обучение модели линейной регрессии на всех данных

lr_full = LinearRegression(labelCol=target, featuresCol='features')
model_full = lr_full.fit(train_data)

In [23]:
# Вывод реальных значений и предсказанных моделью

predictions_full = model_full.transform(test_data)

predicted_full = predictions_full.select("median_house_value", "prediction")
predicted_full.show(5)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0| 152990.3849449535|
|           50800.0| 214967.4326647101|
|           58100.0|142714.18373001367|
|           68400.0|132483.04401091905|
|           72200.0|   164098.87269447|
+------------------+------------------+
only showing top 5 rows



Далее обучим модель на числовых данных, исключив столбец с категориями

In [24]:
# Обучение модели линейной регрессии

lr_part = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')
model_part = lr_part.fit(train_data)

In [25]:
# Вывод реальных значений и предсказанных моделью

predictions_part = model_part.transform(test_data)

predicted_part = predictions_part.select("median_house_value", "prediction")
predicted_part.show(5)

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0| 101397.0241272091|
|           50800.0| 183325.8939412092|
|           58100.0|109609.16753687244|
|           68400.0| 80433.62265761895|
|           72200.0|129998.26626995392|
+------------------+------------------+
only showing top 5 rows



Дальше посчитаем метрики качества моделей (RMSE, MAE, R2) для сравнения моделей

In [26]:
evaluator_rmse = RegressionEvaluator(labelCol="median_house_value",
                                predictionCol="prediction",
                                metricName="rmse")

evaluator_mae = RegressionEvaluator(labelCol="median_house_value",
                                predictionCol="prediction",
                                metricName="mae")

evaluator_r2 = RegressionEvaluator(labelCol="median_house_value",
                                predictionCol="prediction",
                                metricName="r2")


rmse_full = evaluator_rmse.evaluate(predictions_full)
rmse_part = evaluator_rmse.evaluate(predictions_part)

mae_full = evaluator_mae.evaluate(predictions_full)
mae_part = evaluator_mae.evaluate(predictions_part)

r2_full = evaluator_r2.evaluate(predictions_full)
r2_part = evaluator_r2.evaluate(predictions_part)

In [27]:
table = {"RMSE": pd.Series([rmse_full, rmse_part] , index=['Модель, обученная на всех данных', 'Модель, обученная только на числовых переменных']),
        'MAE': pd.Series([mae_full, mae_part], index=['Модель, обученная на всех данных', 'Модель, обученная только на числовых переменных']),
        'R2': pd.Series([r2_full, r2_part], index=['Модель, обученная на всех данных', 'Модель, обученная только на числовых переменных'])}
table = pd.DataFrame(table)
table

,RMSE,MAE,R2
"Модель, обученная на всех данных",68486.730006,49867.047677,0.653558
"Модель, обученная только на числовых переменных",69212.861253,50866.559773,0.646173


## Анализ результатов

Отклонения прогнозируемых значений от реальных в обоих моделях достаточно велики, именно поэтому метрики RMSE и MAE оказались настолько высокими. По сравнениям двух моделей видно, что категориальный столбец все же является значимым и помогает модели немного лучше предсказывать значения медианной стоимости дома. Коэффициент детерминации - это показатель качества модели, который показывает долю дисперсии зависимой переменной, объясняемой рассматриваемой моделью, соответственно чем она выше, тем модель лучше. Модель обученная на всех данных слегка лучше второй модели,  однако если оценивать модели в общем и целом, то они обе не особо надежные, возможно следовало бы добавить еще какие-то факторы для улучшения модели 